<a href="https://colab.research.google.com/github/hayunjong83/computer_vision_implement_research/blob/master/keras_resources/Introduction_to_keras_for_engineers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Keras for engineers

- 엔지니어를 위한 케라스 소개
- 원본 글 : [Introduction to Keras for Engineers](https://keras.io/getting_started/intro_to_keras_for_engineers/)

## Introduction

케라스는 TensorFlow, JAX, PyTorch 위에서 상호 호환적으로 동작할 수 있는 딥러닝 프레임워크다.

## Setup

이 노트북에서는 실행 백엔드로서 JAX를 선택한다.

- 만약 <code>"jax"</code>를 <code>"tensorflow"</code> 또는 <code>"torch"</code>로 변경하면, 다른 백엔드 위에서 실행시킬 수 있다.

In [1]:
import numpy as np
import os

os.environ["KERAS_BACKENDS"] = "jax"

import keras

- 실행환경에서 환경변수 <code>KERAS_BACKENDS</code>를 지정함으로써, 백엔드를 지정할 수 있다. 로컬 환경에서는 다음과 같은 방법으로도 할 수 있다.
    + 쉘에서 환경 변수 지정 : <code>export KERAS_BACKENDS="jax"</code>
    + 케라스 설정 파일( <code>~/.keras/keras.json</code>)에서 환경변수 지정

- 이 때, **반드시 keras를 불러오기(import) 전에**, 백엔드 설정이 이뤄져야 한다.

## 첫 번째 예제 : MNIST convnet

### 데이터셋 준비

- MNIST는 0~9 사이의 숫자가 기록된 디지털 이미지를 분류하기 위한 데이터셋이다.
    + 더 자세한 사항은 [MNIST 홈페이지](https://yann.lecun.com/exdb/mnist/)에서 확인할 수 있다.

- 케라스를 비롯한 딥러닝 프레임워크에서는 새로운 학습자를 위한 데이터셋을 제공한다.
    + 케라스의 내장(built-in) 데이터셋 : [Built-in small datasets](https://keras.io/api/datasets/)
    + <code>keras.datasets.mnist.load_data()</code>를 사용하면, **이미지 데이터**와 대응되는 **정답 레이블**로 구성된 데이터를 받을 수 있다.
    + 수와 크기, 데이터 타입 등 추가 정보는 [MNIST digits classification dataset](https://keras.io/api/datasets/mnist/)을 참고한다.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
print(f"Type of Dataset: {type(x_train)}")
print(f"Shape of dataset: Train-{x_train.shape}, Test-{x_test.shape}")
print(f"Shape of label : Train-{y_train.shape}, Test-{y_test.shape}")

Type of Dataset: <class 'numpy.ndarray'>
Shape of dataset: Train-(60000, 28, 28), Test-(10000, 28, 28)
Shape of label : Train-(60000,), Test-(10000,)


- 60,000장의 훈련데이터와 10,000장의 테스트데이터가 받아진다.
- 데이터는 넘파이 배열(numpy.ndarray)이다.

In [4]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170,
        253, 253, 253, 253, 253, 225, 172, 253, 242, 195,  64,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253, 253, 253, 253,
        253, 253, 253, 253, 251,  93,  82,  82,  56,  39,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  18, 219, 253, 253, 253, 253,
        253, 198, 182, 247, 241,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  80, 156, 107, 253, 253,
        205,  11,   0,  43, 154,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  14,   1, 154, 253,
         90,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 139, 253,
        190,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  11, 190,
        253,  70,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  35,
        241, 225, 160, 108,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         81, 240, 253, 253, 119,  25,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  45, 186, 253, 253, 150,  27,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  16,  93, 252, 253, 187,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 249, 253, 249,  64,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  46, 130, 183, 253, 253, 207,   2,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  39,
        148, 229, 253, 253, 253, 250, 182,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  24, 114, 221,
        253, 253, 253, 253, 201,  78,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  23,  66, 213, 253, 253,
        253, 253, 198,  81,   2,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,  18, 171, 219, 253, 253, 253, 253,
        195,  80,   9,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,  55, 172, 226, 253, 253, 253, 253, 244, 133,
         11,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 136, 253, 25

In [5]:
y_train[0]

5

In [6]:
x_train.dtype

dtype('uint8')

- 위에서 볼 수 있는 예제 데이터로부터, 이미지는 (28 x 28) 크기의 회색(greyscale) 이미지임을 알 수 있다.
- 디지털 이미지는 0부터 255 사이의 정수값(<code>uint8</code>)으로 되어 있다.
- 좀 더 효과적인 학습을 위해서, 정수값을 0부터 1 사이의 실수값으로 바꾸고,
- 높이(H: Height)와 너비(W: Width)로만 이뤄진 각 이미지(=2차원 행렬)를, 채널 차원이 추가된 3차원 행렬(H x W x C)로 변경할 필요가 있다.

In [7]:
x_train, x_test = [x.astype("float32") / 255 for x in [x_train, x_test]]

In [8]:
x_train, x_test = [np.expand_dims(x, -1) for x in [x_train, x_test]]

- 단일 색상인 그레이스케일 이미지인 MNIST 데이터는 각 픽셀값을 채널 차원으로 추가하여 주면 되므로, <code>numpy.expand_dims()</code>를 사용한다.
- <code>numpy.exapnd_dims()</code>에서 <code>axis=-1</code>로 지정하면, 마지막 차원의 데이터를 새로운 차원의 데이터로 확장한다.

In [9]:
x_train.shape

(60000, 28, 28, 1)

### 모델 작성

케라스에서 제공하는 딥러닝 모델 작성방식은 아래와 같다.
- [Sequential API](https://keras.io/guides/sequential_model/)
- [Functional API](https://keras.io/guides/functional_api/)
- [Subclassing을 통해, 직접 모델을 구성](https://keras.io/guides/making_new_layers_and_models_via_subclassing/)

여기에서는 가장 간단한 방식인 Sequential API를 사용한다. 이는 케라스의 layers를 오직 선형으로만 쌓을 수 있지만, 매우 간단하게 모델을 구성할 수 있다.

In [10]:
num_classes = 10
input_shape = (28, 28, 1)

In [11]:
model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

- 이미지 분류에 최적화된 컨볼루션 신경망(CNN, Convolution Neural Network)을 구성하였다.

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 10, 10, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

- 모델의 세부 내용에 관한 요약은 [summary()](https://keras.io/api/models/model/#summary-method) 메소드를 통하여 출력할 수 있다.

### 학습을 위한 설정과 훈련

- 옵티마이저(optimizer), 손실함수(loss function), 모니터할 메트릭(metric)을 지정하기 위해 <code>compile()</code>를 사용한다.
- 추가적인 내용은 Model training APIs의 [compile method](https://keras.io/api/models/model_training_apis/#compile-method)를 참고할 수 있다.
    + 이 문서에 따르면, `compile()`의 `jit_compile` 옵션에 따라, XLA 컴파일러 사용여부를 지정할 수 있다.
    + 따로 지정하지 않은 경우, 디폴트 값인 `auto`는 JAX와 TensorFlow 백엔드 사용시 XLA로 컴파일되지만, PyTorch의 경우에는 컴파일러에 의한 최적화 없이 즉시 실행(eager execution)되는 것으로 보인다.
    + 모델 가속화를 위한 xla(Accelerated Linear Algebra)에 대해서는 [OpenXLA 문서](https://openxla.org/xla)를 참고한다.

In [13]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics = [
        keras.metrics.SparseCategoricalAccuracy(name='acc')
    ]
)

이제 필요한 하이퍼 파라미터 설정을 추가하여, 모델 훈련을 진행한다.

- 여기에서는 훈련 데이터의 15%를 검증셋으로 나눠서, 처음 보는 데이터에 대한 일반화 능력을 모니터링한다.

In [14]:
batch_size=128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
source = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - acc: 0.5342 - loss: 1.3076 - val_acc: 0.9564 - val_loss: 0.1444
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - acc: 0.9283 - loss: 0.2428 - val_acc: 0.9777 - val_loss: 0.0780
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9526 - loss: 0.1594 - val_acc: 0.9827 - val_loss: 0.0619
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9644 - loss: 0.1232 - val_acc: 0.9849 - val_loss: 0.0482
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9688 - loss: 0.1034 - val_acc: 0.9841 - val_loss: 0.0549
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9711 - loss: 0.0957 - val_acc: 0.9887 - val_loss: 0.0421
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - acc: 0.9756 - loss: 0.0823 - val_acc: 0.9903 - val_loss: 0.0374
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - acc: 0.9775 - loss: 0.0730 - val_acc: 0.9863 - val_loss: 0.0463
Epoch 9/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms

- 콜백함수를 이용하여서, 각 에포크의 끝에서 모델을 저장하게 하였다.
- 그러나 모델 훈련이 끝난 후에 다음 명령어를 이용하여, 훈련된 모델을 직접 저장할 수 있다.
```python
model.save("final_model.keras")
```
- 저장된 모델은 다음 명령어를 이용하여 다시 불러올 수 있다.
```python
model = keras.saving.load_model("final_model.keras")
```

In [15]:
prediction = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


- 각 클래스에 대한 확률을 포함한 예측을 수행할 때에는 `predict()`를 이용한다.

## 프레임워크에 상관없이 동작하는 맞춤 요소 작성

- 케라스에서는 백엔드 프레임워크의 종류와 상관없이 동작하는 커스텀 요소들(레이어, 모델, 메트릭, 손실, 옵티마이저)을 작성할 수 있다.
- `keras.ops` 네임스페이스에는 다음 요소들이 있다.
    + NumPy 연산의 실행 : 예) `keras.ops.stack`, `keras.ops.matmul`
    + NumPy에서 제공되지 않는 신경망을 위한 연산들 : 예) `keras.ops.conv`, `keras.ops.binary_crossentropy`

다음은 밀집 레이어(Dense layer)를 직접 작성하는 경우다.

- [`keras.ops.matmul`](https://keras.io/api/ops/numpy/#matmul-function)

In [16]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, name=None):
        super().__init__(name=name)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=(input_dim, self.units),
            initializer=keras.initializers.GlorotNormal(),
            name = "kernel",
            trainable=True,
        )
        self.b = self.add_weight(
            shape = (self.units,),
            initializer =keras.initializers.Zeros(),
            name="bias",
            trainable=True,
        )

    def call(self, inputs):
        x = keras.ops.matmul(inputs, self.w) + self.b
        return self.activation(x)

- `keras.random` 네임스페이스를 이용하여 Dropout을 구현한다.
    + 무작위 연산(random operation)에 대한 내용은 RNG(Random Number Generator) API [문서](https://keras.io/api/random/)에서 찾아볼 수 있다.

In [17]:
class MyDropout(keras.layers.Layer):
    def __init__(self, rate, name=None):
        super().__init__(name=name)
        self.rate = rate
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        return keras.random.dropout(inputs, self.rate, seed = self.seed_generator)

이제 이렇게 정의한 두 레이어를 서브 클래싱을 이용하여, 새로운 모델을 구성하는데 사용할 수 있다.

In [18]:
class MyModel(keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_base = keras.Sequential(
            [
                keras.layers.Conv2D(64, kernel_size = (3, 3), activation="relu"),
                keras.layers.Conv2D(64, kernel_size = (3, 3), activation="relu"),
                keras.layers.MaxPooling2D(pool_size=(2,2)),
                keras.layers.Conv2D(128, kernel_size = (3, 3), activation="relu"),
                keras.layers.Conv2D(128, kernel_size = (3, 3), activation="relu"),
                keras.layers.GlobalAveragePooling2D(),
            ]
        )
        self.dp = MyDropout(0.5)
        self.dense = MyDense(num_classes, activation="softmax")

    def call(self, x):
        x = self.conv_base(x)
        x = self.dp(x)
        return self.dense(x)

In [19]:
model = MyModel(num_classes=10)
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=1,
    validation_split=0.15
)

399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - acc: 0.5368 - loss: 1.2951 - val_acc: 0.9233 - val_loss: 0.2577


## 임의의 데이터 입력형태에 대해 모델 훈련시키기

- 모든 케라스 모델은 다양한 형식의 데이터 포맷에 대해서 훈련 및 평가를 할 수 있으며, 이 역시 어떠한 백엔드를 사용하는지와 상관없다.
    + NumPy 배열
    + Pandas 데이터프레임(dataframe)
    + 텐서플로의 `tf.data.Dataset`
    + 파이토치의 `DataLoader` 객체
    + 케라스의 `PyDataset` 객체

파이토치의 `DataLoaders`를 사용하는 케라스의 방식이다.

In [20]:
import torch

train_torch_dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(x_train), torch.from_numpy(y_train)
)
val_torch_dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(x_test), torch.from_numpy(y_test)
)

train_dataloader = torch.utils.data.DataLoader(
    train_torch_dataset, batch_size = batch_size, shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_torch_dataset, batch_size = batch_size, shuffle=False
)

In [21]:
model = MyModel(num_classes=10)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name='acc'),
    ]
)
model.fit(train_dataloader, epochs=1, validation_data=val_dataloader)

469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - acc: 0.5805 - loss: 1.1815 - val_acc: 0.9375 - val_loss: 0.2050


텐서플로의 `tf.data`를 이용하는 경우다.

In [22]:
import tensorflow as tf

train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [23]:
model = MyModel(num_classes = 10)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name='acc'),
    ]
)
model.fit(train_dataset, epochs=1, validation_data=test_dataset)

469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - acc: 0.5548 - loss: 1.2624 - val_acc: 0.9171 - val_loss: 0.2667
